In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import os
import numpy as np

In [13]:
df1=pd.read_csv('url_result.csv')
df1=df1.drop_duplicates()

In [14]:
df1=df1.loc[:10,:]
df1

,맛집url
0,/restaurants/cTAy9QS_J5cl
1,/restaurants/0kuRscmhbREW
2,/restaurants/0eY8YzMwpZ
3,/restaurants/X2oyP4U6-a
4,/restaurants/iXjQqcw9rdi4
5,/restaurants/1pmv75vVQx
6,/restaurants/extAj1Oeyx
7,/restaurants/WOaP1-KVWysz
8,/restaurants/9VLsn_7CfpNv
9,/restaurants/JUsGTjz2Kq_R


In [15]:
np=np.array(df1['맛집url'].tolist())

In [19]:
driver=webdriver.Chrome('./chromedriver85.exe')
action=ActionChains(driver)

# 망고 전체 크롤링

In [20]:
result=[]
count = 0
for i in np:
    count += 1
    #print (i)
    processpoint = count/len(np) * 100          # processpoint는 repeat 수치를 값으로, 총 횟수 대비 현재 위치 비율이다.
    print( "%0.f%%" % processpoint)

    driver.get('https://www.mangoplate.com/'+str(i))
    html_doc = driver.page_source
    soup = BeautifulSoup(html_doc, 'html.parser')
    #맛집 이름
    title = soup.find("h1",{"class": "restaurant_name"}).text
    #맛집 평점
    rating = soup.find("strong",{"class": "rate-point"})
    # info = soup.find("tbody").get_text().replace('\n', '')
    # print(info)
    
    info = dict()
    if rating and title !=None:
        info['평점'] = rating.text.replace('\n', '')
        info['이름'] = title
    elif rating==None:
        info['평점'] = None
        info['이름'] = title
    else:
        info['평점'] = None
        info['이름'] = None
    table = soup.find("tbody")
    for thtd in table.find_all("tr"):
        temp = thtd.th.get_text().replace(' ', '')
        info[temp.replace('\n', '')] = thtd.td.get_text().replace('\n', '')

    # while True:
    #     try:
    #         더보기 = driver.find_element_by_css_selector('body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > div.RestaurantReviewList__MoreReviewButton')
    #         더보기.click()
    #         time.sleep(2)
    #     except:
    #         break

  
    list=soup.select('body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li > a > div.RestaurantReviewItem__ReviewContent > div')
    reviews=[]
    for i in list:
        review = i.find("p","RestaurantReviewItem__ReviewText").text.replace('\n', '')
        reviews.append(review)
        #print(reviews)
    info['리뷰']=reviews
    
    result.append(info)

9%
18%
27%
36%
45%
55%
64%
73%
82%
91%
100%


In [11]:
#print(result)

In [21]:
name=[]
rate=[]
address=[]
call=[]
food=[]
price=[]
parking=[]
time=[]
menu=[]
reviews=[]
for x in result:
    if '이름' in x.keys():
        name.append(x['이름'])
    else:
        name.append(None)
    if '평점' in x.keys():
        rate.append(x['평점'])
    else:
        rate.append(None)
    if '주소' in x.keys():
        address.append(x['주소'])
    else:
        address.append(None)
    if '전화번호' in x.keys():
        call.append(x['전화번호'])
    else:
        call.append(None)
    if '음식종류' in x.keys():
        food.append(x['음식종류'])
    else:
        food.append(None)  
    if '가격대' in x.keys():
        price.append(x['가격대'])
    else:
        price.append(None)
    if '주차' in x.keys():
        parking.append(x['주차'])
    else:
        parking.append(None) 
    if '영업시간' in x.keys():
        time.append(x['영업시간'])
    else:
        time.append(None)
    if '메뉴' in x.keys():
        menu.append(x['메뉴'])
    else:
        menu.append(None)
    if '리뷰' in x.keys():
        reviews.append(x['리뷰'])
    else:
        reviews.append(None)
    
#     print(x.keys())


In [22]:
data=pd.DataFrame({
    '이름':name,
    '평점':rate,
    '주소':address,
    '전화번호':call,
    '음식종류':food,
    '가격대':price,
    '주차':parking,
    '영업시간':time,
    '메뉴':menu,
     '리뷰':reviews
})

In [23]:
data

,이름,평점,주소,전화번호,음식종류,가격대,주차,영업시간,메뉴,리뷰
0,진미평양냉면,4.6,서울특별시 강남구 학동로 305-3지번서울시 강남구 논현동 115-10,02-515-3469,국수 / 면 요리,만원-2만원,주차공간없음,11:00 - 21:30,"냉면 (물/비빔)11,000원편육 (소고기)28,000원제육26,000원제육반13,...",[ 듣던대로 만두맛집!물냉면이랑 만두 한접시 주문했는데 전 평냉은 ...
1,서관면옥,4.5,서울특별시 서초구 서초대로56길 11지번서울시 서초구 서초동 1675-5,02-521-9945,국수 / 면 요리,만원-2만원,무료주차 가능,11:00 - 22:00,"평양냉면13,000원골동냉면13,000원선비냉면13,000원맛박이냉면15,000원돝...","[ 냉면은 깔끔하고 나머지 구성도 다채롭다 , ..."
2,을지면옥,4.4,서울특별시 중구 충무로 72-1지번서울시 중구 입정동 156,02-2266-7052,국수 / 면 요리,만원-2만원,주차공간없음,11:00 - 21:00,"냉면12,000원비빔냉면12,000원소고기국밥10,000원수육28,000원편육24,...","[ 내 기준 평양냉면 최고 , 3호선..."
3,봉피양,4.4,서울특별시 송파구 양재대로71길 1-4 벽제갈비지번서울시 송파구 방이동 205-8 ...,02-415-5527,고기 요리,만원-2만원,발렛,11:00 - 22:00,"돼지본갈비 (270g)27,000원한우떡갈비 (200g)30,000원평양냉면14,000원",[ 해장으로 평냉은 참 좋다. 봉피양 평냉은 좀 간이 센 편이라고 ...
4,설눈,4.4,서울특별시 서초구 서초대로46길 20-7지번서울시 서초구 서초동 1551-9,02-6959-9339,국수 / 면 요리,만원-2만원,무료주차 가능,11:00 - 21:00,None,[ 소고기 수육 완전 부들부들비빔도 맛있고 물냉도 맛있어요물김치 김...
5,우래옥,4.3,서울특별시 중구 창경궁로 62-29지번서울시 중구 주교동 118-1,02-2265-0151,국수 / 면 요리,만원-2만원,무료주차 가능,11:30 - 21:30,"평양냉면14,000원비빔냉면14,000원불고기 (150g)35,000원육회 (150...",[ 우래옥 불고기에 물냉면 조합은 언제나 옳다. 아삭하고 새콤한 겉...
6,정인면옥,4.4,서울특별시 영등포구 국회대로76길 10지번서울시 영등포구 여의도동 13-1,02-2683-2615,국수 / 면 요리,만원-2만원,무료주차 가능,11:30 - 21:30,"평양냉면 (물/비빔)10,000원한우불고기(1인분)23,000원녹두전8,000원순면...",[ 2020 미슐랭 빕그루망. 정석적이고 깔끔한 맛을 자랑합니다. ...
7,봉래면옥,4.4,서울특별시 강동구 명일로 200-16지번서울시 강동구 길동 227-1,02-6081-2500,국수 / 면 요리,만원 미만,무료주차 가능,11:00 - 22:00,None,"[ 강동구에서 평양냉면을 접할 수 있고, 100% 순메밀면의 선택이..."
8,평안도상원냉면,4.3,서울특별시 마포구 양화로 156지번서울시 마포구 동교동 165-8,02-332-8935,국수 / 면 요리,만원 미만,None,None,,[ 1. 상가건물 지하에있어서 찾기어려울수도있음2. 토요일 점심때갔...
9,태천면옥,4.3,서울특별시 광진구 광장로 49지번서울시 광진구 광장동 258-39,02-466-7224,국수 / 면 요리,만원 미만,None,None,None,[]
